# PrimaCare AI - Competition Submission Notebook
## MedGemma Impact Challenge | All 4 Award Tracks

This notebook is the reproducible submission for the MedGemma Impact Challenge.

**Tracks covered:**
1. **Main Track** — 5-agent CXR pipeline with MedGemma + MedSigLIP, binary pneumonia evaluation (F1 0.803)
2. **Agentic Workflow Prize** — 5 coordinated agents, orchestrator with profiling, RAG guidelines
3. **Novel Task Prize** — PatientEducationAgent: health literacy translation at 3 reading levels
4. **Edge AI Prize** — MedSigLIP ONNX INT8 quantization for CPU-only pneumonia screening

**Architecture:**
```
Patient → IntakeAgent → ImagingAgent → ReasoningAgent → GuidelinesAgent → EducationAgent → Report
                                                                                    ↓
                                                                            Patient-Friendly
                                                                             Education
```

**Tiered deployment:**
```
[Edge - CPU Only]                         [Cloud - GPU]
MedSigLIP ONNX INT8 → Pneumonia? ──Y──→ Full 5-Agent Pipeline
       │                                        │
       └── Normal ─────────────────────→ Done    └──→ Report + Education
```

In [ ]:
# Config
from dataclasses import dataclass

@dataclass
class CompetitionConfig:
    model_id: str = "google/medgemma-1.5-4b-it"
    n_per_class: int = 50
    evaluation_seed: int = 42
    bootstrap_samples: int = 500
    classification_mode: str = "binary"
    threshold_objective: str = "recall_priority"

CFG = CompetitionConfig()
CFG


In [ ]:
import os
import sys
import subprocess
import inspect
import random
import time
from dataclasses import dataclass
from pathlib import Path

os.environ.setdefault("TORCHDYNAMO_DISABLE", "1")

# ---------------------------------------------------------------------
# Auth bootstrap (fail fast for gated MedGemma access)
# ---------------------------------------------------------------------

def _resolve_hf_token():
    token = os.environ.get("HF_TOKEN")
    if token:
        return token

    try:
        from kaggle_secrets import UserSecretsClient
        token = UserSecretsClient().get_secret("HF_TOKEN")
        if token:
            return token
    except Exception:
        pass

    return None

def _login_and_verify_medgemma(token):
    from huggingface_hub import HfApi, login

    login(token=token, add_to_git_credential=False)
    os.environ["HF_TOKEN"] = token

    try:
        HfApi().model_info("google/medgemma-1.5-4b-it", token=token)
    except Exception as exc:
        raise RuntimeError(
            "HF_TOKEN found, but access to google/medgemma-1.5-4b-it is not available. "
            "Accept model terms at https://huggingface.co/google/medgemma-1.5-4b-it and ensure the token has access."
        ) from exc

hf_token = _resolve_hf_token()
if not hf_token:
    raise RuntimeError(
        "Missing HF_TOKEN. In Kaggle, add it under Add-ons -> Secrets as key HF_TOKEN, "
        "then rerun from the first cell."
    )

_login_and_verify_medgemma(hf_token)
print("HF authentication OK and MedGemma access verified.")

# ---------------------------------------------------------------------
# Repo bootstrap
# ---------------------------------------------------------------------

def _is_repo_root(path: Path) -> bool:
    return (path / "src" / "agents" / "orchestrator.py").exists()

def _find_repo_root():
    candidates = [
        Path.cwd(),
        Path("/kaggle/working/Med-Gemma"),
        Path("/kaggle/input/med-gemma"),
        Path("/kaggle/input/med-gemma-repo"),
        Path("/kaggle/input/med-gemma-impact-challenge"),
    ]

    for candidate in candidates:
        if _is_repo_root(candidate):
            return candidate

    input_root = Path("/kaggle/input")
    if input_root.exists():
        for d in input_root.iterdir():
            if d.is_dir() and _is_repo_root(d):
                return d

    return None

repo_root = _find_repo_root()

if repo_root is None:
    clone_target = Path("/kaggle/working/Med-Gemma")
    try:
        subprocess.run(
            ["git", "clone", "https://github.com/thestai-admin/Med-Gemma.git", str(clone_target)],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
        )
    except Exception as exc:
        raise RuntimeError(
            "Could not locate local src/ package and GitHub clone failed. "
            "Attach the repo as a Kaggle Dataset or enable internet for clone."
        ) from exc

    if not _is_repo_root(clone_target):
        raise RuntimeError("Repository cloned but src/agents/orchestrator.py not found.")

    repo_root = clone_target

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print("Using repo root:", repo_root)

import numpy as np
import torch
from datasets import load_dataset

from src.agents import PrimaCareOrchestrator

# ---------------------------------------------------------------------
# Inline deterministic evaluation helpers (self-contained; no src.eval)
# ---------------------------------------------------------------------

@dataclass
class EvalMetrics:
    threshold: float
    n_samples: int
    accuracy: float
    precision: float
    recall: float
    specificity: float
    f1: float
    tp: int
    tn: int
    fp: int
    fn: int

    def to_dict(self):
        return {
            "threshold": self.threshold,
            "n_samples": self.n_samples,
            "accuracy": self.accuracy,
            "precision": self.precision,
            "recall": self.recall,
            "specificity": self.specificity,
            "f1": self.f1,
            "tp": self.tp,
            "tn": self.tn,
            "fp": self.fp,
            "fn": self.fn,
        }

@dataclass
class ThresholdResult:
    threshold: float
    metrics: EvalMetrics

@dataclass
class LatencyMetrics:
    runs: int
    raw_timings: dict
    median_by_stage: dict
    p95_by_stage: dict

def _safe_div(a, b):
    return float(a) / float(b) if b else 0.0

def compute_binary_metrics(y_true, y_pred, threshold=0.5):
    y_true = np.asarray(y_true, dtype=np.int32)
    y_pred = np.asarray(y_pred, dtype=np.int32)
    tp = int(np.sum((y_true == 1) & (y_pred == 1)))
    tn = int(np.sum((y_true == 0) & (y_pred == 0)))
    fp = int(np.sum((y_true == 0) & (y_pred == 1)))
    fn = int(np.sum((y_true == 1) & (y_pred == 0)))

    precision = _safe_div(tp, tp + fp)
    recall = _safe_div(tp, tp + fn)
    specificity = _safe_div(tn, tn + fp)
    accuracy = _safe_div(tp + tn, len(y_true))
    f1 = _safe_div(2 * precision * recall, precision + recall)

    return EvalMetrics(
        threshold=float(threshold),
        n_samples=int(len(y_true)),
        accuracy=accuracy,
        precision=precision,
        recall=recall,
        specificity=specificity,
        f1=f1,
        tp=tp,
        tn=tn,
        fp=fp,
        fn=fn,
    )

def sweep_thresholds(y_true, scores, thresholds=None):
    if thresholds is None:
        thresholds = np.arange(0.10, 0.91, 0.05)
    scores = np.asarray(scores, dtype=np.float32)
    y_true = np.asarray(y_true, dtype=np.int32)

    out = []
    for t in thresholds:
        y_pred = (scores >= float(t)).astype(np.int32)
        metrics = compute_binary_metrics(y_true, y_pred, threshold=float(t))
        out.append(ThresholdResult(threshold=float(t), metrics=metrics))
    return out

def select_threshold(results, objective="balanced"):
    if not results:
        raise ValueError("No threshold results provided.")
    objective = objective.strip().lower()
    if objective == "recall_priority":
        return max(results, key=lambda r: (r.metrics.recall, r.metrics.f1, r.metrics.specificity))
    return max(results, key=lambda r: (r.metrics.f1, r.metrics.recall, r.metrics.specificity))

def bootstrap_metric_ci(y_true, y_pred, metric="f1", n_bootstrap=500, seed=42, alpha=0.95):
    y_true = np.asarray(y_true, dtype=np.int32)
    y_pred = np.asarray(y_pred, dtype=np.int32)
    rng = np.random.default_rng(seed)

    vals = []
    n = len(y_true)
    for _ in range(int(n_bootstrap)):
        idx = rng.integers(0, n, n)
        m = compute_binary_metrics(y_true[idx], y_pred[idx])
        vals.append(getattr(m, metric))

    lo = float(np.quantile(vals, (1.0 - alpha) / 2.0))
    hi = float(np.quantile(vals, 1.0 - ((1.0 - alpha) / 2.0)))
    return lo, hi

def _supports_kwarg(fn, name):
    try:
        return name in inspect.signature(fn).parameters
    except Exception:
        return False

def profile_orchestrator_latency(orchestrator, run_kwargs, repeats=3):
    raw = {}
    for _ in range(int(repeats)):
        kwargs = dict(run_kwargs)
        if _supports_kwarg(orchestrator.run, "profile"):
            kwargs["profile"] = True

        start = time.perf_counter()
        result = orchestrator.run(**kwargs)
        elapsed = time.perf_counter() - start

        timings = getattr(result, "timings", None) or {}
        if not timings:
            timings = {"total": elapsed}

        for k, v in timings.items():
            raw.setdefault(k, []).append(float(v))

    median_by_stage = {k: float(np.median(v)) for k, v in raw.items()}
    p95_by_stage = {k: float(np.quantile(v, 0.95)) for k, v in raw.items()}
    return LatencyMetrics(
        runs=int(repeats),
        raw_timings=raw,
        median_by_stage=median_by_stage,
        p95_by_stage=p95_by_stage,
    )

def analyze_image_compat(orchestrator, image, classification_mode):
    analyze_fn = orchestrator.imaging_agent.analyze
    kwargs = {
        "image": image,
        "include_classification": True,
    }
    if _supports_kwarg(analyze_fn, "classification_mode"):
        kwargs["classification_mode"] = classification_mode
    if _supports_kwarg(analyze_fn, "skip_classification_if_confident"):
        kwargs["skip_classification_if_confident"] = False
    return analyze_fn(**kwargs)

random.seed(CFG.evaluation_seed)
np.random.seed(CFG.evaluation_seed)
torch.manual_seed(CFG.evaluation_seed)

print(torch.__version__)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
# Initialize orchestrator
orchestrator = PrimaCareOrchestrator(enable_guidelines=True)

# Warm-up image-only call (optional)
# _ = orchestrator.analyze_image(sample_image, include_classification=False, profile=False)


## Evaluation Protocol

- Use held-out or explicitly separated samples.
- Keep class balance explicit in reported tables.
- Report confusion counts and uncertainty intervals.


In [ ]:
def collect_balanced_pneumonia_samples(n_per_class=50):
    ds = load_dataset("hf-vision/chest-xray-pneumonia", split="train", streaming=True)
    normal, pneumonia = [], []
    for sample in ds:
        label = int(sample["label"])
        if label == 0 and len(normal) < n_per_class:
            normal.append(sample)
        elif label == 1 and len(pneumonia) < n_per_class:
            pneumonia.append(sample)
        if len(normal) >= n_per_class and len(pneumonia) >= n_per_class:
            break
    return normal + pneumonia

samples = collect_balanced_pneumonia_samples(CFG.n_per_class)
len(samples)


In [ ]:
# Score extraction for binary/ensemble modes
y_true = []
scores = []

for sample in samples:
    image = sample["image"].convert("RGB")
    y_true.append(int(sample["label"]))

    analysis = analyze_image_compat(
        orchestrator=orchestrator,
        image=image,
        classification_mode=CFG.classification_mode,
    )

    score = 0.0
    if hasattr(analysis, "classification_probs"):
        probs = analysis.classification_probs or {}
        if "pneumonia" in probs:
            score = float(probs["pneumonia"])
        elif probs:
            # fallback: use max non-normal probability if exact key absent
            normal_like = [k for k in probs.keys() if "normal" in k.lower()]
            normal_score = max([probs[k] for k in normal_like], default=0.0)
            score = float(max(0.0, 1.0 - normal_score))

    if score == 0.0 and hasattr(orchestrator.imaging_agent, "classify_pneumonia_binary"):
        bp = orchestrator.imaging_agent.classify_pneumonia_binary(image)
        score = float(bp.get("pneumonia", 0.0))

    scores.append(float(score))

results = sweep_thresholds(y_true, scores)
best = select_threshold(results, objective=CFG.threshold_objective)
metrics = best.metrics

pred = [1 if s >= best.threshold else 0 for s in scores]
f1_ci = bootstrap_metric_ci(y_true, pred, metric="f1", n_bootstrap=CFG.bootstrap_samples, seed=CFG.evaluation_seed)

print("Selected threshold:", best.threshold)
print(metrics.to_dict())
print("F1 95% CI:", f1_ci)


In [ ]:
# Latency profile (use one representative case)
sample_case = {
    "chief_complaint": "cough",
    "history": "65yo with productive cough and dyspnea",
    "xray_image": samples[0]["image"].convert("RGB"),
    "include_classification": True,
}

# pass classification_mode only if this orchestrator version supports it
if _supports_kwarg(orchestrator.run, "classification_mode"):
    sample_case["classification_mode"] = CFG.classification_mode

latency = profile_orchestrator_latency(orchestrator, run_kwargs=sample_case, repeats=3)
latency.median_by_stage


## Track 3: Novel Task Prize — Patient Education Agent

The PatientEducationAgent converts technical clinical reports into patient-friendly language at 3 reading levels, addressing the health literacy gap that affects ~36% of US adults.

**Reading Levels:**
- **Basic** — 6th-grade vocabulary, no medical jargon
- **Intermediate** — Common medical terms with explanations
- **Detailed** — Full clinical terminology with definitions

Each output includes a **glossary** of medical terms for patient reference.

In [ ]:
# --- Track 3: Patient Education Demo ---
# Run full pipeline with education enabled on a sample CXR case

sample_image = samples[0]["image"].convert("RGB")

print("Running full 5-agent pipeline with patient education enabled...")
print("=" * 60)

result_with_education = orchestrator.run(
    chief_complaint="Cough for 2 weeks with fever",
    history="65 year old male smoker. Productive cough with yellow sputum. Low-grade fever. Night sweats.",
    xray_image=sample_image,
    age=65,
    gender="male",
    include_education=True,
    education_level="basic",
    profile=True,
)

# Show the education section from the full report
print("\n" + "=" * 60)
print("PATIENT EDUCATION OUTPUT (Basic Level)")
print("=" * 60)
if result_with_education.patient_education:
    print(result_with_education.patient_education.to_report_section())
    print("\n--- Glossary ---")
    for term, defn in result_with_education.patient_education.glossary.items():
        print(f"  {term}: {defn}")
else:
    print("Education not generated (model may not have returned structured output)")

# Show pipeline timings including education step
print("\n--- Pipeline Timings (with education) ---")
for stage, t in sorted(result_with_education.timings.items()):
    print(f"  {stage}: {t:.2f}s")
print(f"\nProcessing steps: {result_with_education.processing_steps}")

In [ ]:
# --- Compare all 3 reading levels ---
from src.agents.education import PatientEducationAgent

education_agent = PatientEducationAgent(model=orchestrator.model)

print("Generating patient education at all 3 reading levels...")
print("=" * 60)

for level in ["basic", "intermediate", "detailed"]:
    education = education_agent.educate(result_with_education, reading_level=level)
    print(f"\n{'='*60}")
    print(f"READING LEVEL: {level.upper()}")
    print(f"{'='*60}")
    print(f"\nDiagnosis: {education.simplified_diagnosis[:200]}...")
    print(f"\nGlossary terms: {list(education.glossary.keys())}")
    print(f"Total glossary entries: {len(education.glossary)}")

## Track 4: Edge AI Prize — CPU-Only Pneumonia Screening

The Edge AI module exports MedSigLIP to ONNX and quantizes to INT8 for CPU-only deployment.

**Tiered architecture:**
- **Edge tier (CPU):** Fast binary pneumonia screening — runs in clinics without GPU
- **Cloud tier (GPU):** Full 5-agent pipeline for cases flagged by edge screening

**Process:**
1. Export MedSigLIP vision encoder → ONNX FP32
2. Quantize → INT8 dynamic (onnxruntime)
3. Pre-compute text embeddings for binary labels ("normal" / "pneumonia")
4. At inference: run vision encoder on CPU, cosine similarity with cached text embeddings

In [ ]:
# --- Track 4: Edge AI — Export, Quantize, and Benchmark ---
import os
from pathlib import Path

edge_dir = Path("models/edge")
edge_dir.mkdir(parents=True, exist_ok=True)

fp32_path = str(edge_dir / "medsiglip_fp32.onnx")
int8_path = str(edge_dir / "medsiglip_int8.onnx")

# Step 1: Export MedSigLIP vision encoder to ONNX
print("=" * 60)
print("Step 1: Exporting MedSigLIP to ONNX (FP32)")
print("=" * 60)
from src.edge.quantize import export_medsiglip_onnx, quantize_onnx_int8

export_medsiglip_onnx(fp32_path)

# Step 2: Quantize to INT8
print("\n" + "=" * 60)
print("Step 2: Quantizing to INT8")
print("=" * 60)
quantize_onnx_int8(fp32_path, int8_path)

# Report file sizes
fp32_size = os.path.getsize(fp32_path) / (1024 * 1024)
int8_size = os.path.getsize(int8_path) / (1024 * 1024)
print(f"\nFP32 model: {fp32_size:.1f} MB")
print(f"INT8 model: {int8_size:.1f} MB")
print(f"Reduction:  {(1 - int8_size/fp32_size)*100:.1f}%")

In [ ]:
# --- Edge Classifier: CPU-only inference ---
from src.edge.inference import EdgeClassifier
from src.edge.benchmark import run_edge_benchmark, EdgeBenchmarkResult, compare_models

print("Loading INT8 edge classifier (CPU only)...")
edge_classifier = EdgeClassifier(int8_path)
print(f"Edge model size: {edge_classifier.model_size_mb:.1f} MB")

# Classify a few sample images on CPU
print("\n--- Edge CPU Classification Results ---")
for i in range(min(5, len(samples))):
    img = samples[i]["image"].convert("RGB")
    label = "pneumonia" if samples[i]["label"] == 1 else "normal"
    result = edge_classifier.classify_pneumonia(img)
    pred = "pneumonia" if result["pneumonia"] > result["normal"] else "normal"
    match = "OK" if pred == label else "MISS"
    print(f"  Sample {i}: true={label:10s} pred={pred:10s} "
          f"P(pneumonia)={result['pneumonia']:.3f}  [{match}]")

In [ ]:
# --- Edge Benchmark: Latency + Accuracy on evaluation set ---
# Use first 20 samples for benchmark (full set is slow on CPU)
n_bench = min(20, len(samples))
bench_images = [s["image"].convert("RGB") for s in samples[:n_bench]]
bench_labels = [int(s["label"]) for s in samples[:n_bench]]

print(f"Running edge benchmark on {n_bench} samples...")
edge_result = run_edge_benchmark(
    edge_classifier, bench_images, bench_labels,
    model_type="edge_int8",
)

print("\n" + "=" * 60)
print("EDGE BENCHMARK RESULTS")
print("=" * 60)
for k, v in edge_result.to_dict().items():
    print(f"  {k:<25} {v}")

# Compare with GPU model (construct result from earlier evaluation)
gpu_result = EdgeBenchmarkResult(
    model_type="gpu_full",
    model_size_mb=3500.0,  # MedSigLIP full model
    avg_latency_ms=latency.median_by_stage.get("imaging", 15.0) * 1000,
    p95_latency_ms=latency.p95_by_stage.get("imaging", 20.0) * 1000,
    memory_peak_mb=4000.0,  # ~4GB VRAM
    accuracy=metrics.accuracy,
    recall=metrics.recall,
    f1=metrics.f1,
    n_samples=metrics.n_samples,
)

print("\n" + compare_models(gpu_result, edge_result))

## Summary: All 4 Award Tracks

| Track | Prize | Key Evidence |
|-------|-------|-------------|
| **Main Track** | $75K | 5-agent CXR pipeline, Binary F1 0.803, 100 samples, bootstrap CI |
| **Agentic Workflow** | $10K | 5 coordinated agents, orchestrator profiling, RAG guidelines, parallel execution |
| **Novel Task** | $10K | PatientEducationAgent: 3 reading levels, medical glossary, health literacy |
| **Edge AI** | $5K | MedSigLIP ONNX INT8, CPU-only inference, benchmark comparison |

**Models used:** MedGemma 1.5 4B (multimodal), MedSigLIP 448 (classification), all-MiniLM-L6-v2 (RAG embeddings)

**Tests:** 42 passing, 1 skipped (GPU) — all tests run with mocks, no GPU needed

**Code:** [github.com/thestai-admin/Med-Gemma](https://github.com/thestai-admin/Med-Gemma)

---

*PrimaCare AI is clinician decision support, not autonomous diagnosis. All outputs require verification by qualified healthcare professionals.*